In [5]:
import pandas as pd
df = pd.read_csv('demo_events.csv')
display(df.head(2))
df.shape

,Subject,Start date,Location,source,Start time,End time
0,Buntes Herbstfest im Tierpark,10/04/2024,"Tierpark Neumünster | 24537 Neumünster, Geerdt...",https://city-nms.de/events/veranstaltungskalen...,10:00,NaN
1,Die Spur des Bildhauers. Wiedersehen mit Heinz...,10/04/2024,"Herbert Gerisch-Stiftung | 24536 Neumünster, B...",https://city-nms.de/events/veranstaltungskalen...,12:00,18:00


(6, 6)

In [6]:
# Uploading data to a BigQuery database. This is a better approach, because it uses batch requets.
import os.path
from datetime import timedelta
import pandas as pd
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import BatchHttpRequest

from google.cloud import bigquery
from pandas_gbq import to_gbq

# Define the combined scopes
SCOPES = [
    "https://www.googleapis.com/auth/calendar",      # Google Calendar
    "https://www.googleapis.com/auth/bigquery"       # BigQuery
]

def authenticate_google_services():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    # Return the Google Calendar service along with creds.
    service = build('calendar', 'v3', credentials=creds)
    return service, creds

service, creds = authenticate_google_services()  # Authenticate and get credentials for Google Calendar and BigQuery.

project_id = 'circular-maxim-436817-e3'
dataset_id = 'kierne35539432nun34b'
table_id = 'events'
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

client = bigquery.Client(credentials=creds, project=project_id)

try:
    to_gbq(df, destination_table=full_table_id, project_id=project_id, if_exists='replace', credentials=creds, progress_bar=True)
    print("Data uploaded to BigQuery successfully!")   #  Inserting data into BigQuery "events" table.
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded to BigQuery successfully!


In [7]:
def delete_all_events_in_batches(service, calendarId='primary', batch_size=1000):
    """Deletes all events from Google Calendar using batch requests and tracks successful deletions, handling pagination."""
    
    total_deleted = 0  # Track the total number of successfully deleted events
    page_token = None  # To handle pagination
    
    # Callback function to track each successful deletion
    def callback(request_id, response, exception):
        nonlocal total_deleted
        if exception is None:
            total_deleted += 1  # Increment on successful deletion
    
    # Fetch and delete events page by page
    while True:
        # Fetch events with pagination
        events_result = service.events().list(calendarId=calendarId, pageToken=page_token).execute()
        events = events_result.get('items', [])
        
        if not events:
            break  # No more events to process

        event_ids = [event['id'] for event in events]  # Extract event IDs
        total_events_in_page = len(event_ids)

        # Process the events in batches for each page of results
        for i in range(0, total_events_in_page, batch_size):
            batch = BatchHttpRequest(batch_uri="https://www.googleapis.com/batch/calendar/v3")
            current_batch_ids = event_ids[i:i+batch_size]
            
            for event_id in current_batch_ids:
                batch.add(service.events().delete(calendarId=calendarId, eventId=event_id), callback=callback)
            
            # Execute the batch deletion
            batch.execute()
            print(f"Processed batch {i // batch_size + 1} with {len(current_batch_ids)} events in a current page")
        
        # Check if there's a next page of events
        page_token = events_result.get('nextPageToken')
        if not page_token:
            break  # No more pages

    print(f"Total of {total_deleted} events were deleted successfully from Google Calendar.")

delete_all_events_in_batches(service)

Processed batch 1 with 6 events in current page
Total of 6 events were deleted successfully.


In [10]:
def load_events_from_csv():
    """Loads events from a CSV DataFrame and returns them as a list of event objects."""
    events = []

    query = f"SELECT * FROM {full_table_id}"
    df_loaded = client.query(query).to_dataframe()  # Load the data from BigQuery into a pandas DataFrame.
    print('The shape of loaded dataframe is:', df_loaded.shape)

    for _, row in df_loaded.iterrows():
        # Convert start date and time to datetime
        start_datetime_str = f"{row['Start date']}T{row['Start time']}"
        start_time_obj = pd.to_datetime(start_datetime_str)

        # Check if End time is provided; if not, add 27 minutes to Start time
        if pd.isna(row.get('End time')):  # If 'End time' is None or NaN
            end_time_obj = start_time_obj + timedelta(minutes=27)  # Add 27 minutes
        else:
            # Convert End time and assume same date as Start date
            end_datetime_str = f"{row['Start date']}T{row['End time']}"
            end_time_obj = pd.to_datetime(end_datetime_str)
        
        # Prepare the event details for Google Calendar
        event = {
            'summary': row['Subject'],
            'location': row['Location'],
            'start': {
                'dateTime': start_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                'timeZone': 'Europe/Berlin',  # Set to your timezone
            },
            'end': {
                'dateTime': end_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                'timeZone': 'Europe/Berlin',  # Set to your timezone
            }
        }
        events.append(event)
    print("The total of", len(events), "events were loaded from BigQuery")
    return events

events = load_events_from_csv() 

The shape of loaded dataframe is: (6, 6)
The total of 6 events were loaded from BigQuery


In [11]:
# Inserting events to Google Calendar.
event_ids = []  # Initialize a list to store event IDs

def callback(request_id, response, exception):
    if exception is not None:
        # Log or handle the error accordingly
        print(f"Error: {exception}")
    else:
        event_id = response.get('id')  # Get the event ID from the response
        event_ids.append(event_id)  # Append the event ID to the list
        print(f"Event created: {response.get('htmlLink')}") # should be deleted.

# Function to create events in batches
def create_events_in_batches(service, events, batch_size=1000):
    """Creates events in Google Calendar using multiple batch requests."""
    total_events = len(events)
    
    # Create a batch request object with the correct batch path for Google Calendar API
    batch = BatchHttpRequest(callback=callback, batch_uri="https://www.googleapis.com/batch/calendar/v3")
    
    for i, event in enumerate(events):
 
        # Add the request to the batch using only the relative path for each event insert request
        request = service.events().insert(calendarId='primary', body=event)
        batch.add(request, request_id=str(i))
        
        # Execute the batch when the batch_size is reached or when at the last event
        if (i + 1) % batch_size == 0 or (i + 1) == total_events:
            print(f"Executing batch {i // batch_size + 1} for events {i - batch_size + 1} to {i + 1}...")
            batch.execute()  # Execute the batch
            print(f"Processed batch {i // batch_size + 1}")

            # Reset the batch for the next set of events
            batch = BatchHttpRequest(callback=callback, batch_uri="https://www.googleapis.com/batch/calendar/v3")
    
create_events_in_batches(service, events)
print(f"Total events successfully created in Calendar: {len(event_ids)}") 

Executing batch 1 for events -994 to 6...
Event created: https://www.google.com/calendar/event?eid=OWFtMDh2Y3Ryb2p1NW5nZHYwZnVjM2Njb2cgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=aTNxYTM0YnYxNXV2cW5paW80a2wzamozZjAgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=aG0xa25rdW00ZnNyYXUwNWVya2gxb2Q4MDQgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=dG9nZ2Y4NzhubGV1Z2p0bmFhMDdnYnBsaDAgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=azY5cmU0NnRqZmY3Z2kwNWo5a25vZmRiZm8gZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=dmpjMmNtdTBhMW11cmNuOG5mY2RqYjc0MDAgZm9rczI0M2RhYmw0QG0
Processed batch 1
Total events successfully created: 6
